In [1]:
import sys
sys.path.append("/home/stachu/Projects/Anomaly_detection/Forecasting_models")

from predpy.dataset import MultiTimeSeriesDataset
from predpy.data_module import MultiTimeSeriesModule
from predpy.wrapper import Autoencoder, Predictor, VAE
from predpy.experimentator import (
    DatasetParams, ModelParams,
    Experimentator, load_experimentator)
from predpy.plotter import (
    plot_exp_predictions, plot_anomalies
)
from predpy.preprocessing import set_index
from predpy.preprocessing import moving_average
from predpy.preprocessing import (
    load_and_preprocess, set_index, moving_average, drop_if_is_in,
    use_dataframe_func, loc, iloc, get_isoforest_filter, get_variance_filter)
from predpy.trainer import (
    CheckpointParams, TrainerParams, EarlyStoppingParams, LoggerParams)
from tsad.noiser import apply_noise_on_dataframes, white_noise
from tsad.anomaly_detector import PredictionAnomalyDetector, ReconstructionAnomalyDetector
from models import LSTMAE, LSTMVAE
from literature.AnomTrans import AnomalyTransformer, ATWrapper

from pytorch_lightning.loggers import TensorBoardLogger
import pickle
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# from tsai.models import TCN, ResNet, TST, RNN, TransformerModel, FCN
import pandas as pd
from torch.utils.data import DataLoader, Dataset

# Template

In [2]:
# window_size = 366

# load_params = {
#     "sep": ';', "header": 0, "low_memory": False,
#     "infer_datetime_format": True, "parse_dates": {'datetime': [0, 1]},
#     "index_col": ['datetime']
# }

# columns = ["Global_active_power"]  # , "Voltage"]
# drop_refill_pipeline = [
#     (loc, {"columns": columns}),
#     (drop_if_is_in, (["?", np.nan]), {"columns": columns}),
#     (iloc, {"rows_end": 5000}),
#     # (iloc, {"rows_start": -20000}),
# ]
# preprocessing_pipeline = [
#     (use_dataframe_func, "astype", "float"),
# ]
# detect_anomalies_pipeline = [
#     # (get_isoforest_filter, dict(
#     #     scores_threshold=-0.36, window_size=500, target="Global_active_power"))
#     (get_variance_filter, dict(
#         window_size=3000, log_variance_limits=(-7, 0),
#         target="Global_active_power"))
# ]


# datasets_params = [
#     DatasetParams(
#         path="/home/stachu/Projects/Anomaly_detection/TSAD/data/Energy/household_power_consumption/household_power_consumption.csv",
#         load_params=load_params,
#         target="Global_active_power",
#         split_proportions=[0.8, 0.1, 0.1],
#         window_size=window_size,
#         batch_size=64,
#         drop_refill_pipeline=drop_refill_pipeline,
#         preprocessing_pipeline=preprocessing_pipeline,
#         detect_anomalies_pipeline=detect_anomalies_pipeline,
#         scaler=StandardScaler(),
#         resample_params=dict(
#             resampler_method_str="fillna", rule="1min", resample_kwargs={},
#             resampler_method_kwargs=dict(method="backfill")),
# ]

# c_in = 1
# c_out = 1

# models_params = [
#     # # ModelParams(
#     # #     name_="TST_l3_fcDrop0.1", cls_=TST.TST,
#     # #     init_params={
#     # #         "c_in": c_in, "c_out": c_out, "seq_len": window_size,
#     # #         "max_seq_len": window_size, "n_layers": 3, "fc_dropout": 0.1}),
#     # # ModelParams(
#     # #     name_="TST_l2_fcDrop0.1", cls_=TST.TST,
#     # #     init_params={
#     # #         "c_in": c_in, "c_out": c_out, "seq_len": window_size,
#     # #         "max_seq_len": window_size, "n_layers": 2, "fc_dropout": 0.1}),
#     # ModelParams(
#     #     name_="TST_l2_fcDrop0.0", cls_=TST.TST,
#     #     init_params={
#     #         "c_in": c_in, "c_out": c_out, "seq_len": window_size,
#     #         "max_seq_len": window_size, "n_layers": 2, "fc_dropout": 0.0}),
#     # ModelParams(
#     #     name_="ResNet", cls_=ResNet.ResNet,
#     #     init_params={"c_in": c_in, "c_out": c_out}),
#     # # ModelParams(
#     # #     name_="LSTM_h200_l1", cls_=RNN.LSTM,
#     # #     init_params={
#     # #         "c_in": c_in, "c_out": c_out, "hidden_size": 200, "n_layers": 1}),
#     # # ModelParams(
#     # #     name_="LSTM_h200_l2", cls_=RNN.LSTM,
#     # #     init_params={
#     # #         "c_in": c_in, "c_out": c_out, "hidden_size": 200, "n_layers": 2}),
#     # ModelParams(
#     #     name_="LSTM_h400_l2", cls_=RNN.LSTM,
#     #     init_params={
#     #         "c_in": c_in, "c_out": c_out, "hidden_size": 400, "n_layers": 2}),
#     # ModelParams(
#     #     name_="LSTMAutoencoder_h200_l1", cls_=LSTMAE,
#     #     init_params=dict(
#     #         c_in=window_size, h_size=200, n_layers=1),
#     #     WrapperCls=Autoencoder),
#     ModelParams(
#         name_="LSTMVAE_h200_l1", cls_=LSTMVAE,
#         init_params=dict(
#             c_in=window_size, h_size=200, n_layers=1),
#         WrapperCls=VAE, wrapper_kwargs=dict(kld_weight=0.005)),
#     # ModelParams(
#     #     name_="LSTMVAE_h200_l1", cls_=LSTMVAE,
#     #     init_params=dict(
#     #         c_in=window_size, h_size=200, n_layers=1),
#     #     WrapperCls=VAE, wrapper_kwargs=dict(kld_weight=0.002)),
# ]

# chp_p = CheckpointParams(
#     dirpath="./checkpoints", monitor='val_loss', verbose=True,
#     save_top_k=1)
# tr_p = TrainerParams(
#     max_epochs=1, gpus=1, auto_lr_find=True)
# es_p = EarlyStoppingParams(
#     monitor='val_loss', patience=2, verbose=True)

# # import pytorch_lightning as pl
# # from pytorch_lightning.loggers import TensorBoardLogger

# # tmp = pl.Trainer(logger=TensorBoardLogger("./"))

# exp = Experimentator(
#     models_params=models_params,
#     datasets_params=datasets_params,
#     trainer_params=tr_p,
#     checkpoint_params=chp_p,
#     early_stopping_params=es_p,
#     LoggersClasses=[TensorBoardLogger],
#     loggers_params=[LoggerParams(save_dir="./lightning_logs")]
# )

# exp.run_experiments(experiments_path="./saved_experiments", safe=False)
# # exp = load_experimentator(
# #     "./saved_experiments/2021-12-29_01:31:42.pkl"
# # )

# plot_exp_predictions(exp, dataset_idx=0, models_ids=[0])


# household_power_consumption

## Dataset

In [3]:
window_size = 366

load_params = {
    "sep": ';', "header": 0, "low_memory": False,
    "infer_datetime_format": True, "parse_dates": {'datetime': [0, 1]},
    "index_col": ['datetime']
}

columns = ["Global_active_power"]  # , "Voltage"]
drop_refill_pipeline = [
    (loc, {"columns": columns}),
    (drop_if_is_in, (["?", np.nan]), {"columns": columns}),
    (iloc, {"rows_end": 5000}),
    # (iloc, {"rows_start": -20000}),
]
preprocessing_pipeline = [
    (use_dataframe_func, "astype", "float"),
]
detect_anomalies_pipeline = [
    # (get_isoforest_filter, dict(
    #     scores_threshold=-0.36, window_size=500, target="Global_active_power"))
    (get_variance_filter, dict(
        window_size=3000, log_variance_limits=(-7, 0),
        target="Global_active_power"))
]


datasets_params = [
    DatasetParams(
        path="/home/stachu/Projects/Anomaly_detection/TSAD/data/Energy/household_power_consumption/household_power_consumption.csv",
        load_params=load_params,
        target="Global_active_power",
        split_proportions=[0.8, 0.1, 0.1],
        window_size=window_size,
        batch_size=64,
        drop_refill_pipeline=drop_refill_pipeline,
        preprocessing_pipeline=preprocessing_pipeline,
        detect_anomalies_pipeline=detect_anomalies_pipeline,
        scaler=StandardScaler(),
        resample_params=dict(
            resampler_method_str="fillna", rule="1min", resample_kwargs={},
            resampler_method_kwargs=dict(method="backfill")))
]

## Models

In [4]:
c_in = 1
c_out = 1

models_params = [
    ModelParams(
        name_="AnomTrans", cls_=AnomalyTransformer,
        init_params=dict(
            N=window_size, d_model=1, layers=2, lambda_=0.5),
        WrapperCls=ATWrapper),
    # ModelParams(
    #     name_="LSTMVAE_h200_l1", cls_=LSTMVAE,
    #     init_params=dict(
    #         c_in=window_size, h_size=200, n_layers=1),
    #     WrapperCls=VAE, wrapper_kwargs=dict(kld_weight=0.005)),
]

## Experiments

In [5]:
chp_p = CheckpointParams(
    dirpath="./checkpoints", monitor='val_loss', verbose=True,
    save_top_k=1)
tr_p = TrainerParams(
    max_epochs=1, gpus=1, auto_lr_find=True)
es_p = EarlyStoppingParams(
    monitor='val_loss', patience=2, verbose=True)

exp = Experimentator(
    models_params=models_params,
    datasets_params=datasets_params,
    trainer_params=tr_p,
    checkpoint_params=chp_p,
    early_stopping_params=es_p,
    LoggersClasses=[TensorBoardLogger],
    loggers_params=[LoggerParams(save_dir="./lightning_logs")]
)

## Run / Load

In [6]:
exp.run_experiments(experiments_path="./saved_experiments", safe=False)
# exp = load_experimentator(
#     "./saved_experiments/2021-12-29_01:31:42.pkl"
# )

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/stachu/.cache/pypoetry/virtualenvs/tsad-8szBw8Wl-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:59: LightningDeprecationWarning:

Setting `Trainer(flush_logs_every_n_steps=50)` is deprecated in v1.5 and will be removed in v1.7. Please configure flushing in the logger instead.

/home/stachu/.cache/pypoetry/virtualenvs/tsad-8szBw8Wl-py3.8/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning:

DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | model     | AnomalyTransformer | 20    
1 | criterion | MSELoss            | 0  

Validation sanity check:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: ('Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!', 'Problem with training 0 model named AnomTrans on 0 dataset named household_power_consumption')

## Plot

In [ ]:
plot_exp_predictions(exp, dataset_idx=0, models_ids=[0])

# ServerMachineDataset

## Dataset

In [ ]:
window_size = 100

load_params = {
    "header": None
}

# columns = ["Global_active_power"]  # , "Voltage"]
drop_refill_pipeline = [
    # (loc, {"columns": columns}),
    # (drop_if_is_in, (["?", np.nan]), {"columns": columns}),
    # (iloc, {"rows_end": 5000}),
    # (iloc, {"rows_start": -20000}),
]
preprocessing_pipeline = [
    (use_dataframe_func, "astype", "float"),
]
detect_anomalies_pipeline = [
    # (get_isoforest_filter, dict(
    #     scores_threshold=-0.36, window_size=500, target="Global_active_power"))
    # (get_variance_filter, dict(
    #     window_size=3000, log_variance_limits=(-7, 0),
    #     target="Global_active_power"))
]


datasets_params = [
    DatasetParams(
        path="/home/stachu/Projects/Anomaly_detection/TSAD/data/Industry/ServerMachineDataset/train/machine-1-1.csv",
        load_params=load_params,
        target=list(range(38)),
        split_proportions=[0.8, 0.1, 0.1],
        window_size=window_size,
        batch_size=64,
        drop_refill_pipeline=drop_refill_pipeline,
        preprocessing_pipeline=preprocessing_pipeline,
        detect_anomalies_pipeline=detect_anomalies_pipeline,
        scaler=StandardScaler()),
]

## Models

In [ ]:
c_in = 38
c_out = 38

models_params = [
    ModelParams(
        name_="LSTMVAE_h200_l1", cls_=LSTMVAE,
        init_params=dict(
            c_in=window_size, h_size=200, n_layers=1),
        WrapperCls=VAE, wrapper_kwargs=dict(kld_weight=0.005)),
    ModelParams(
        name_="LSTMVAE_h200_l1", cls_=LSTMVAE,
        init_params=dict(
            c_in=window_size, h_size=200, n_layers=1),
        WrapperCls=VAE, wrapper_kwargs=dict(kld_weight=0.005)),
]

## Experiments

In [ ]:
chp_p = CheckpointParams(
    dirpath="./checkpoints", monitor='val_loss', verbose=True,
    save_top_k=1)
tr_p = TrainerParams(
    max_epochs=5, gpus=1, auto_lr_find=True)
es_p = EarlyStoppingParams(
    monitor='val_loss', patience=2, verbose=True)

exp = Experimentator(
    models_params=models_params,
    datasets_params=datasets_params,
    trainer_params=tr_p,
    checkpoint_params=chp_p,
    early_stopping_params=es_p,
    LoggersClasses=[TensorBoardLogger],
    loggers_params=[LoggerParams(save_dir="./lightning_logs")]
)

## Run / Load

In [ ]:
exp.run_experiments(experiments_path="./saved_experiments", safe=False)
# exp = load_experimentator(
#     "./saved_experiments/2021-12-29_01:31:42.pkl"
# )

/home/stachu/Projects/Anomaly_detection/TSAD/predpy/preprocessing/load_and_preprocess.py:188: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/home/stachu/Projects/Anomaly_detection/TSAD/predpy/preprocessing/load_and_preprocess.py:203: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

Global seed set to 42
GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/stachu/.cache/pypoetry/virtualenvs/tsad-8szBw8Wl-py3.8/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:59: LightningDeprecationWarning:

Setting `Trainer(flush_logs_every_n_steps=50)` is deprecated in v1.5 and will be removed in v1.7. Please configure flushing in the logger instead.

/home/stachu/.cache/pypoetry/virtualenvs/tsad-8szBw8Wl-py3.8/lib/python3.8/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning:

DataModule.setup ha

Global seed set to 42


Epoch 0: 100%|██████████| 398/398 [00:11<00:00, 34.80it/s, loss=0.417, v_num=5:08, train_loss=0.124, val_loss=0.935]

Metric val_loss improved. New best score: 0.935


Epoch 0: 100%|██████████| 398/398 [00:11<00:00, 34.78it/s, loss=0.417, v_num=5:08, train_loss=0.124, val_loss=0.935]

...

Epoch 4: 100%|██████████| 398/398 [00:08<00:00, 44.96it/s, loss=0.193, v_num=5:08, train_loss=0.0744, val_loss=0.563]

Epoch 4, global step 1764: val_loss reached 0.56304 (best 0.56304), saving model to "/home/stachu/Projects/Anomaly_detection/TSAD/scripts/checkpoints/machine-1-1/LSTMVAE_h200_l1/2022-04-11_19:25:08.ckpt" as top 1


Epoch 4: 100%|██████████| 398/398 [00:08<00:00, 44.85it/s, loss=0.193, v_num=5:08, train_loss=0.0744, val_loss=0.563]


Making predictions: 100%|██████████| 45/45 [00:01<00:00, 31.79it/s]


Experiments ended sucessfully


## Plot

In [ ]:
plot_exp_predictions(exp, dataset_idx=0, models_ids=[0])
# plot_anomalies(exp.datasets_params.iloc[0])